## 1.引言

多头注意力是指将注意力机制分为多个“头”，每个头学习数据的不同方面，使模型能够从多个层面来捕获输入序列中各部分之间的关系，这提高了模型在复杂任务中的表现。

本文将在上篇文章[因果注意力]()的基础上，一步一步解读多头注意力的简单实现方法和高效实现方法。

## 2.堆叠实现

多头注意力最直接的实现方法是将多个单头注意力的实例堆叠在一起，如下图所示:
![多头堆叠](./img/7-1.jpg)

 > 图中有两个头的多头注意力模块，每个头都有自己的权重矩阵Wq/Wk/Wv，都有自己中间状态Q/K/V，最后合并两组上下文向量Z1/Z2得到上下文向量矩阵Z。

 我们通过将前面已经实现的CausalAttention进行堆叠来演示。首先运行已经封装好的`attention_v1.py`（前文的因果注意力已经封装在里面）。

In [2]:
%run attention_v1.py

定义一个多头注意力类MultiHeadAttentionStack，构造函数与CausalAttention的唯一区别是多了一个num_heads入参，用来指定多头的数量。

In [3]:
from torch import nn

class MultiHeadAttentionStack(nn.Module):
    def __init__(self, dim_in, dim_out, context_length, dropout_rate, num_heads):
        super().__init__()
        multi_attens = [CausalAttention(dim_in, dim_out, context_length, dropout_rate) for i in range(num_heads)]
        self.heads = nn.ModuleList(multi_attens)

    def forward(self, x):
        context_vecs = [head(x) for head in self.heads]
        return torch.cat(context_vecs, dim=-1)

使用刚封装的多头注意力来计算上下文向量，输出维度设为了2，但由于num_heads=2指定了使用两个头，所以最终的上下文向量是dim_out * num_heads = 4 维。

In [8]:
torch.manual_seed(123)
batch = torch.stack((inputs, inputs), dim=0)
b, token_len, dim = batch.shape 
multi_heads = MultiHeadAttentionStack(dim, 2, token_len, 0.0, num_heads=2)
context_vecs = multi_heads(batch)
context_vecs

tensor([[[0.1855, 0.8812, 1.3211, 0.8098],
         [0.3116, 0.9549, 1.6063, 1.1493],
         [0.3395, 0.9652, 1.6530, 1.2084],
         [0.3129, 0.8747, 1.5012, 1.0955],
         [0.2865, 0.7897, 1.4100, 1.0398],
         [0.2990, 0.8040, 1.4025, 1.0361]],

        [[0.1855, 0.8812, 1.3211, 0.8098],
         [0.3116, 0.9549, 1.6063, 1.1493],
         [0.3395, 0.9652, 1.6530, 1.2084],
         [0.3129, 0.8747, 1.5012, 1.0955],
         [0.2865, 0.7897, 1.4100, 1.0398],
         [0.2990, 0.8040, 1.4025, 1.0361]]], grad_fn=<CatBackward0>)

> 批量输入张量batch中有两个输入文本，由于两个文本是经过复制而来，因此最终的两个文本的上下文张量完全相同。

这样，我们就通过创建并组合多个CausalAttention类，实现了第一个版本的多头注意力模块。但这个实现方式有两个缺点：
1. 需要关心并维护MultiHeadAttentionStack和CausalAttention两个概念，不够简洁。
2. 多个头在计算注意力时需要按顺序以串行的方式，相当于有多少个头就要计算多少遍注意力，不够高效。

为此，我们接下来将尝试改进这个实现。

## 3. 权重分割实现

**权重分割的基本思想**：将大权重矩阵按照头的数量（num_heads）分割，即能模拟出多头各有一个小的权重矩阵的效果。通过这些小的权重矩阵并行的对输入进行变换，就能一次性计算出所有注意力头的输出。
![权重分割](./img/7-2.png)

> 如上图所示，通过初始化一个大的权重矩阵`Wq`，只和输入张量`X`进行一次矩阵乘法来获得查询矩阵`Q`，然后使用view方法对`Q`进行形状重塑即可得到多个头。

下面我们来进行代码实现。

#### 3.1 多头注意力类

由于多头注意力的代码相对稍长，我们分成构造方法和forward方法两部分来说明。

In [4]:
import torch
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, dim_in, dim_out, context_length, dropout_rate, num_heads, qkv_bias=False):
        super().__init__()
        assert dim_out % num_heads == 0, "dim_out must be divisible by num_heads"
        self.dim_out = dim_out
        self.num_heads = num_heads
        self.head_dim = dim_out // num_heads   # 每个头的维度
        self.Wq = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.Wk = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.Wv = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.Wo = nn.Linear(dim_out, dim_out)
        self.dropout = nn.Dropout(dropout_rate)
        
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

构造方法`__init__`与CausalAttention中的实现基本类似，只作了两个小的扩展：
1. 进行了单头维数head_dim的计算，num_heads是多头数量，dim_out是总的输出维度，dim_out/num_heads就是单个头的维度。
2. 增加了一个输出投影层（self.Wo），这个并不是必需，但在许多大语言模型中都常见，所以这里也加进来。

下面对forward方法进行封装，由于需要对矩阵进行分割和重组，所以会涉及到张量的变形（view）和转置（transpose）操作，需要仔细理解每一步运算过后矩阵形状的变化。

In [5]:
def forward(self, x):
    # 输入形状
    b, num_tokens, dim_in = x.shape
    # 求Q\K\V矩阵，形状变为： b, num_tokens, dim_out
    q = self.Wq(x)
    k = self.Wk(x)
    v = self.Wv(x)

    # 变换形状，将最后一维拆成多头，每个头有head_dim维，矩阵形状由三维变为四维。
    q = q.view(b, num_tokens, self.num_heads, self.head_dim)
    k = k.view(b, num_tokens, self.num_heads, self.head_dim)
    v = v.view(b, num_tokens, self.num_heads, self.head_dim)
    
    # 交换第2维和第3维，这一步过后，形状变为：b, num_heads, num_tokens, head_dim
    q = q.transpose(1, 2)   
    k = k.transpose(1, 2)
    v = v.transpose(1, 2)

    # 计算注意力分数，形状变为: b, num_heads, num_tokens, num_tokens
    atten_scores = q @ k.transpose(2, 3)   
    atten_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
    atten_weights = torch.softmax(atten_scores/k.shape[-1]**0.5, dim=-1)
    atten_weights = self.dropout(atten_weights)

    # 计算上下文张量
    context_vecs = atten_weights @ v   # shape: b, num_heads, num_tokens, head_dim
    context_vecs = context_vecs.transpose(1,2)  # shape: b, num_tokens, num_heads, head_dim
    context_vecs = context_vecs.contiguous().view(b, num_tokens, self.dim_out)
    output = self.Wo(context_vecs)

    return output

setattr(MultiHeadAttention, "forward", forward)

几个关注步骤的理解:
1. 关键操作是将dim_out维度分割为num_heads和head_dim，这个分割操作由view方法实现，将维度为(b, num_tokens, dim_out)的张量q、k、v重塑为维度(b, num_tokens, num_heads, head_dim),即将一个大的head切割成了多头。
2. 随后进行了张量转置操作，将多头维度num_heads排在了序列长度num_tokens之前，张量形状变为（b, num_heads, num_tokens, head_dim），正是由于这个转置，使得多个头可以一次性进行批量矩阵乘法，一次性得出所有上下文向量。
3. 最后将两个维度num_heads和num_tokens恢复位置，形状变回(b, num_tokens, num_heads, head_dim)，再通过view操作将多个头的上下文向量合并，向量被重塑（扁平化）成 (b, num_tokens, dim_out) 的形状。

> contiguous方法：张量在某些复杂操作（例如转置、切片）之后可能会变得不连续，而view方法需要连续的内存才能正常工作，contiguous方法的作用就是确保返回一个在内存中连续的张量。

#### 3.2 批量矩阵乘法
上面的q\k\v矩阵在进行权重分割后变成了四维张量，在pytorch中进行四维张量乘法时，矩阵乘法运算将会在最后两个维度之间进行，等价于各个头内部分别运算，最后再堆叠到一起。运算过程如下：

In [9]:
a = torch.tensor([[[[0.1855, 0.8812, 1.3211, 0.8098],
                 [0.3116, 0.9549, 1.6063, 1.1493],
                 [0.3395, 0.9652, 1.6530, 1.2084]],
                  
                 [[0.3129, 0.8747, 1.5012, 1.0955],
                 [0.2865, 0.7897, 1.4100, 1.0398],
                 [0.2990, 0.8040, 1.4025, 1.0361]]]])

a.shape

torch.Size([1, 2, 3, 4])

In [ ]:
我们可以把上面这个矩阵看作是拥有两个头、每个头有3行4列的多头张量。

In [11]:
first_head = a[0,0,:,:]
second_head = a[0,1,:,:]
print("first_head:", first_head)
print("second_head:", second_head)

first_head: tensor([[0.1855, 0.8812, 1.3211, 0.8098],
        [0.3116, 0.9549, 1.6063, 1.1493],
        [0.3395, 0.9652, 1.6530, 1.2084]])
second_head: tensor([[0.3129, 0.8747, 1.5012, 1.0955],
        [0.2865, 0.7897, 1.4100, 1.0398],
        [0.2990, 0.8040, 1.4025, 1.0361]])


In [ ]:
分别对每个头进行矩阵转置后的乘法运算。

In [12]:
first_head_res = first_head @ first_head.T
second_head_res = second_head @ second_head.T
print("first_head_res: ", first_head_res)
print("second_head_res:", second_head_res)

first_head_res:  tensor([[3.2120, 3.9520, 4.0759],
        [3.9520, 4.9100, 5.0715],
        [4.0759, 5.0715, 5.2395]])
second_head_res: tensor([[4.3167, 4.0362, 4.0373],
        [4.0362, 3.7750, 3.7754],
        [4.0373, 3.7754, 3.7763]])


上面的过程可以视为一个批量矩阵乘法运算，等于是将张量`a`和转置了最后两个维度的张量a的视图之间进行批量矩阵乘法，整个过程可以简单用一句代码表示：

In [10]:
a @ a.transpose(2,3)

tensor([[[[3.2120, 3.9520, 4.0759],
          [3.9520, 4.9100, 5.0715],
          [4.0759, 5.0715, 5.2395]],

         [[4.3167, 4.0362, 4.0373],
          [4.0362, 3.7750, 3.7754],
          [4.0373, 3.7754, 3.7763]]]])

可以看到，这个结果和上面多个头单独运算的结果完全相同。



#### 3.3 多头注意力使用
下面用之前两个序列的输入来示例MultiHeadAttention的使用。

In [21]:
torch.manual_seed(123)
b, num_tokens, dim_in = batch.shape
multi_heads = MultiHeadAttention(dim_in, 4, num_tokens, 0.0, 2)
context_vecs = multi_heads(batch)
context_vecs

tensor([[[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0147,  0.3259, -0.0734, -0.3721],
         [-0.0116,  0.3138, -0.0708, -0.3624],
         [-0.0117,  0.2973, -0.0698, -0.3543],
         [-0.0132,  0.2990, -0.0689, -0.3490]],

        [[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0147,  0.3259, -0.0734, -0.3721],
         [-0.0116,  0.3138, -0.0708, -0.3624],
         [-0.0117,  0.2973, -0.0698, -0.3543],
         [-0.0132,  0.2990, -0.0689, -0.3490]]], grad_fn=<ViewBackward0>)

> 虽然 MultiHeadAttention 类看起来比 MultiHeadAttentionStack 更复杂，但它更高效。原因在于我们每一步都只需要一次矩阵乘法，例如`k = self.Wk(x)`执行一遍就可以计算出所有键，这对查询、值、注意力权重等都适用。与之相对的是，在 MultiHeadAttentionStack 中，我们需要按头的数量重复这一矩阵乘法，这在计算上是最昂贵的步骤之一。

**小结**：本节中我们首先通过堆叠多个因果注意力演示了什么是多头，然后通过权重分割和形状变换的方式实现了一个高效的MultiHeadAttention类，并对多维下批量矩阵乘法的运算规则作了说明，这个多头注意力类后面将会用于训练大语言模型。